In [12]:
!pip install emp-orderly emp-orderly-types eth-typeshed-py pandas-ta numpy==1.26.4 matplotlib setuptools

In [13]:
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, plot_heatmaps,
    EMA, SMA, SLOPE, CHOP,
    EmpyrealOrderlySDK,
)
from emp_orderly_types import PerpetualAssetType, Interval, OrderType

load_dotenv()

False

In [14]:
wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [17]:
class SmaCross(Strategy):
    n1: int = 10
    n2: int = 40

    @classmethod
    def update_lags(cls, n1, n2):
        cls.n1 = n1
        cls.n2 = n2

    def init(self):
        close = self.data.close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy(size=0.5)
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell(size=0.5)


tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(SmaCross)
await tester.load_data(
    lookback=14,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.DOGE,
)

# backtest
tester.backtest()

Start                     2024-09-25 04:15:00
End                       2024-10-02 03:05:00
Duration                      6 days 22:50:00
Exposure Time [%]                   65.401897
Equity Final [$]               1101734.002226
Equity Peak [$]                1107937.002966
Return [%]                            10.1734
Buy & Hold Return [%]                3.462458
Return (Ann.) [%]                 8212.886623
Volatility (Ann.) [%]             3008.165655
Sharpe Ratio                         2.730198
Sortino Ratio                    17315.033762
Calmar Ratio                      2611.318377
Max. Drawdown [%]                   -3.145111
Avg. Drawdown [%]                   -0.605906
Max. Drawdown Duration        1 days 18:45:00
Avg. Drawdown Duration        0 days 03:03:00
# Trades                                   33
Win Rate [%]                        48.484848
Best Trade [%]                       9.514144
Worst Trade [%]                     -2.324643
Avg. Trade [%]                    

In [16]:
# plot
tester.plot(show_price_data=False)
plt.show()

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/ademola/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.


[246510:246510:0100/000000.453335:ERROR:zygote_linux.cc(678)] write: Broken pipe (32)
